# M&A Data Preparation – Main Notebook (Template)\n\nThis notebook is a **thin UI wrapper** around the `ma_migration` library.\n\nGuidelines:\n- ✅ Do config + run pipelines + show results\n- ✅ Run YAML contracts and review failures\n- ❌ Do not add business logic here\n

In [ ]:
from pathlib import Path\nfrom ma_migration.core.types import SourceConfig\nfrom ma_migration.pipelines.customers_breakdown import run_from_sources\nfrom ma_migration.contracts.loader import load_contract\nfrom ma_migration.contracts.runner import run_contract\n

## 1) Configure sources\nUpdate these paths/sheets/headers per acquisition.\n

In [ ]:
cfg = SourceConfig(\n    customers_masterfile_path=Path('INPUT/Customers_MasterFile.xlsx'),\n    finishing_methods_path=Path('INPUT/FinishingMethods.xlsx'),\n    pieces_in_circulation_path=Path('INPUT/PiecesInCirculation.txt'),\n    lockers_wearers_ref_path=Path('INPUT/PcsInCirculation_LockersWearers_Ref.txt'),\n    active_garments_glob='INPUT/ActiveGarments/*.xlsx',\n)\n

## 2) Run pipeline\nUse `strict=False` during exploration and `strict=True` before Go-Live.\n

In [ ]:
outputs = run_from_sources(cfg, strict=False)\noutputs.keys()\n

## 3) Validate outputs using YAML contracts\nContracts live under `contracts/`.\n

In [ ]:
contract = load_contract('contracts/mato/customer_master.yaml')\nreport = run_contract(outputs['customer_master'], contract, contract_name='contracts/mato/customer_master.yaml', strict=False)\nreport.summary()\n

In [ ]:
for rr in report.results:\n    if rr.failed_rows is not None:\n        print(rr.rule_id, rr.severity, rr.message)\n        display(rr.failed_rows.head(10))\n